In [ ]:
import openai
import numpy as np
from langchain.document_loaders import PyPDFLoader, TextLoader  
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Function to load documents from various sources
def load_documents(loaders):
    """Load all documents from the provided loaders."""
    documents = []
    for loader in loaders:
        documents.extend(loader.load())
    return documents

# 1. Load and Embed Football Knowledge Base
def setup_knowledge_base():
    """Initialize the knowledge base by loading and embedding documents."""
    loaders = [
        PyPDFLoader("path/to/nfl_rulebook.pdf"),
        TextLoader("path/to/football_analytics_article.txt"),
        # Add paths to the rest of your 15 books and data sources
    ]
    
    # Load all documents
    documents = load_documents(loaders)
    
    # Initialize embeddings and vectorstore
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents, embeddings)
    
    return vectorstore

# 2. Set up ChatGPT API and QA Chain
def setup_qa_chain(vectorstore, temperature=0.7):
    """Set up the GPT-based QA retrieval chain using OpenAI and LangChain."""
    llm = OpenAI(temperature=temperature)  # Control creativity with temperature
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm, 
        chain_type="stuff", 
        retriever=vectorstore.as_retriever()
    )
    return qa_chain

# 3. Play-Calling Algorithm (Advanced)
def call_play(game_situation, opponent_data, player_data, win_probabilities, qa_chain):
    """
    Suggest an optimal play based on game situation, opponent tendencies, 
    player stats, and win probabilities using the knowledge base.
    """
    # Construct a detailed query with relevant data
    query = (
        f"Given the game situation: {game_situation}, "
        f"opponent data: {opponent_data}, "
        f"player data: {player_data}, "
        f"and win probabilities: {win_probabilities}, "
        "what play would you recommend?"
    )
    # Run the query against the retrieval QA system
    response = qa_chain.run(query)
    return response

# 4. Main Interaction Loop
def main():
    """Main loop for user interaction to call plays or answer questions."""
    # Step 1: Setup the knowledge base and QA chain
    vectorstore = setup_knowledge_base()
    qa_chain = setup_qa_chain(vectorstore)

    print("Welcome to the NFL AI Coach! Type 'exit' to quit.")
    
    while True:
        user_input = input("Enter your query or game situation: ")
        
        # Exit condition
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        
        # Simulate play-calling if user specifies a game situation
        if "call play" in user_input.lower():
            # Example placeholders - replace with actual game/opponent/player data
            game_situation = "3rd and 5, ball on opponent's 20-yard line"
            opponent_data = "The defense is weak against screen passes"
            player_data = "RB has been highly effective on short runs"
            win_probabilities = 65.2  # Example win probability data

            # Call the play based on the provided data
            response = call_play(game_situation, opponent_data, player_data, win_probabilities, qa_chain)
        else:
            # If it's a general query, use the QA chain
            response = qa_chain.run(user_input)
        
        print(f"AI Response: {response}")

# Run the interaction loop
if __name__ == '__main__':
    main()
